In [28]:
import requests
import pandas as pd
import json 
import time
import numpy as np 

In [ ]:
# date_range = pd.date_range('2022-01-01', '2024-10-30') # 전체 기간 데이터 조회
# date_range = pd.date_range('2023-04-29', '2024-10-30') # 위 기간 실행 중 2023.4.29에 오류 발생해서 28일까지 데이터는 파일로 저장하고 다시 실행
date_range = pd.date_range('2023-06-27', '2024-10-30')  # 위 기간 실행 중 2023.6.26에 오류 발생해서 25일까지 데이터는 파일로 저장하고 다시 실행
                                                        # 2023.06.26 데이터는 없음 - 27일 데이터부터 조회
# print(date_range)

service_key = '445754734a6c656f363548676f6665'
start, end = 1, 1000
all_df_bikes = []
for date in date_range:
    
    print("data at {0} will be processed".format(date))

    while True:        
        date_string = date.strftime('%Y-%m-%d')
        url = 'http://openapi.seoul.go.kr:8088/{0}/json/tbCycleUseStatus/{1}/{2}/{3}'.format(service_key, start, end, date_string)

        # API 요청
        response = requests.get(url)

        # 응답 상태 확인
        if response.status_code == 200:

            try:
                # 요청이 성공했을 경우 내용 출력
                # print(response.content)
                decoded_response_content = response.content.decode('utf-8')

                dict_content = json.loads(decoded_response_content) 

                df_bike = pd.DataFrame(dict_content['useStatus']['row'])
                all_df_bikes.append(df_bike)

                total_count = int(dict_content['useStatus']['list_total_count'])
                if total_count > end:
                    start = end + 1
                    if total_count > end + 1000:
                        end = end + 1000
                    else:
                        end = total_count
                else:
                    start, end = 1, 1000
                    break

            except Exception as e:
                print(e)
                time.sleep(1)                
                
        else:
            # 오류가 발생했을 경우 상태 코드 출력
            print(f"Request failed with status code: {response.status_code}")




In [ ]:
# 데이터 테스트 
len(all_df_bikes)
all_df_bikes[-2]

In [48]:
len(all_df_bikes)

1476

In [ ]:
# 데이터 병합
merged_df_bikes = pd.concat(all_df_bikes)

In [ ]:
# 데이터 병합 테스트
print( merged_df_bikes.shape )
merged_df_bikes.head()

In [ ]:
# 데이터 csv로 저장
# 하나의 파일로 저장되어야하지만, 다운로드과정에 오류가 있어서 기간별로 저장
# merged_df_bikes.to_csv("../bike-data/bike-data-20220101-20230428.csv", index=False)
# merged_df_bikes.to_csv("../bike-data/bike-data-20230429-20230625.csv", index=False)
merged_df_bikes.to_csv("../bike-data/bike-data-20230627-20241030.csv", index=False)